In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
import pandas as pd
from pyspark.sql.functions import col, when, dayofmonth, from_unixtime, expr
from pyspark.sql.functions import unix_timestamp


In [15]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [16]:
df_pandas = pd.read_parquet("fhv_tripdata_2019-10.parquet")

In [17]:
output_path = "fhv_tripdata_2019-10_modified.parquet"
df_pandas.to_parquet(output_path)

In [18]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.LongType(), True),
    types.StructField('dropOff_datetime', types.LongType(), True),
    types.StructField('PUlocationID', types.DoubleType(), True),
    types.StructField('DOlocationID', types.DoubleType(), True),
    types.StructField('SR_Flag', types.IntegerType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True)
])

In [19]:
#spark_df = spark.createDataFrame(df_pandas, schema)

In [20]:
#spark_df.head()

In [21]:
#spark.conf.set("spark.sql.parquet.int96AsTimestamp", "true")
spark.conf.set("spark.sql.session.timeZone", "GMT")


In [22]:
df = spark.read \
    .option("header", "true") \
    .option("timeZone", "GMT") \
    .schema(schema) \
    .parquet("fhv_tripdata_2019-10_modified.parquet")

In [23]:
df = df.withColumn("time_difference_seconds", (df["dropOff_datetime"]-df["pickup_datetime"])/ 1000000000)
df = df.withColumn("pickup_datetime", from_unixtime(df.pickup_datetime / 1000000000))  # Convert nanoseconds to seconds
df = df.withColumn("dropOff_datetime", from_unixtime(df.dropOff_datetime / 1000000000))  # Convert nanoseconds to seconds

from pyspark.sql.functions import month, dayofmonth, hour

df = df.withColumn("pickup_month", month("pickup_datetime")) \
       .withColumn("pickup_day", dayofmonth("pickup_datetime")) \
       .withColumn("pickup_hour", hour("pickup_datetime"))

In [24]:
from pyspark.sql.functions import substring, concat, lit
# Extract year part
year_part = lit("2019")

# Replace month with 10
month_part = lit("-10")

# Extract day and time parts
day_time_part = substring(df["dropOff_datetime"], 8, 15)

# Concatenate corrected year, month, day, and time parts
new_dropOff_datetime = concat(year_part, month_part, day_time_part)

# Replace the original dropOff_datetime column with the corrected values
df_corrected = df.withColumn("dropOff_datetime", new_dropOff_datetime)


In [25]:
df_corrected = df_corrected.withColumn("dropOff_unix_timestamp", unix_timestamp(df_corrected["dropOff_datetime"], "yyyy-MM-dd HH:mm:ss"))
df_corrected = df_corrected.withColumn("pickup_unix_timestamp", unix_timestamp(df_corrected["pickup_datetime"], "yyyy-MM-dd HH:mm:ss"))
df_corrected = df_corrected.withColumn("time_difference_hours", (df_corrected["dropOff_unix_timestamp"]-df_corrected["pickup_unix_timestamp"])/3600)

In [26]:
df_zones = spark.read \
    .option("header", "true") \
    .csv("taxi+_zone_lookup.csv")

In [27]:
df_zones.head(10)

[Row(LocationID='1', Borough='EWR', Zone='Newark Airport', service_zone='EWR'),
 Row(LocationID='2', Borough='Queens', Zone='Jamaica Bay', service_zone='Boro Zone'),
 Row(LocationID='3', Borough='Bronx', Zone='Allerton/Pelham Gardens', service_zone='Boro Zone'),
 Row(LocationID='4', Borough='Manhattan', Zone='Alphabet City', service_zone='Yellow Zone'),
 Row(LocationID='5', Borough='Staten Island', Zone='Arden Heights', service_zone='Boro Zone'),
 Row(LocationID='6', Borough='Staten Island', Zone='Arrochar/Fort Wadsworth', service_zone='Boro Zone'),
 Row(LocationID='7', Borough='Queens', Zone='Astoria', service_zone='Boro Zone'),
 Row(LocationID='8', Borough='Queens', Zone='Astoria Park', service_zone='Boro Zone'),
 Row(LocationID='9', Borough='Queens', Zone='Auburndale', service_zone='Boro Zone'),
 Row(LocationID='10', Borough='Queens', Zone='Baisley Park', service_zone='Boro Zone')]

In [30]:
# Perform full join on PUlocationID and LocationID
joined_df = df_corrected.join(df_zones, df_corrected["PUlocationID"] == df_zones["LocationID"])

# Drop duplicate columns (if any)
joined_df = joined_df.drop(df_zones["LocationID"])

# Show the resulting DataFrame
joined_df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------------------+------------+----------+-----------+----------------------+---------------------+---------------------+-------+---------------+------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|time_difference_seconds|pickup_month|pickup_day|pickup_hour|dropOff_unix_timestamp|pickup_unix_timestamp|time_difference_hours|Borough|           Zone|service_zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+-----------------------+------------+----------+-----------+----------------------+---------------------+---------------------+-------+---------------+------------+
|              B00009|2019-10-01 00:23:00|2019-10-01 00:35:00|       264.0|       264.0|   null|                B00009|                  720.0|     

In [32]:
joined_df.head(10)

[Row(dispatching_base_num='B00009', pickup_datetime='2019-10-01 00:23:00', dropOff_datetime='2019-10-01 00:35:00', PUlocationID=264.0, DOlocationID=264.0, SR_Flag=None, Affiliated_base_number='B00009', time_difference_seconds=720.0, pickup_month=10, pickup_day=1, pickup_hour=0, dropOff_unix_timestamp=1569890100, pickup_unix_timestamp=1569889380, time_difference_hours=0.2, Borough='Unknown', Zone='NV', service_zone='N/A'),
 Row(dispatching_base_num='B00013', pickup_datetime='2019-10-01 00:11:29', dropOff_datetime='2019-10-01 00:13:22', PUlocationID=264.0, DOlocationID=264.0, SR_Flag=None, Affiliated_base_number='B00013', time_difference_seconds=113.0, pickup_month=10, pickup_day=1, pickup_hour=0, dropOff_unix_timestamp=1569888802, pickup_unix_timestamp=1569888689, time_difference_hours=0.03138888888888889, Borough='Unknown', Zone='NV', service_zone='N/A'),
 Row(dispatching_base_num='B00014', pickup_datetime='2019-10-01 00:11:43', dropOff_datetime='2019-10-01 00:37:20', PUlocationID=264.

In [33]:
joined_df.registerTempTable('joined')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [38]:
df_less_frequent_pickup_location = spark.sql("""
SELECT 
    Zone,
    count(Zone) AS zone_count
FROM
    joined
GROUP BY
    Zone
ORDER BY
    zone_count
LIMIT
    1
""")

In [39]:
df_less_frequent_pickup_location.head(10)

[Row(Zone='Jamaica Bay', zone_count=1)]